In [1]:
import torch
import numpy as np
import os, sys
from pycls.core.config import cfg
import yaml
import pycls.core.model_builder as model_builder
from al_utils.data import Data as custom_Data
from al_utils.ActiveLearning import ActiveLearning
from helper.subprocess_utils import vaal_sampling_util

HOME_DIRECTORY=os.path.abspath(os.path.join(os.getcwd(), os.pardir))

os.chdir(HOME_DIRECTORY)

# This notebook assumes installation of gdown.
# If not, kindly install it via pip i.e pip install gdown

In [5]:
!gdown https://drive.google.com/drive/folders/102nZ5pCofBDncKDFQc9tnyvQ8__IcIOi -O ./notebooks --folder

Retrieving folder list
Retrieving folder 1EMjHz45aAFtlfO348WMECI-_mRSw9Hcf active_set_indexes
Retrieving folder 1buawUbiI5FCG-9aYSBBElRr3RCM4q_cI lSet_1
Retrieving folder 1r7forT9Uzmp7cgwjOGVJokac2wR-LNRS start_1
Retrieving folder 1Hf65hSIUXmPaNnW0VU75aaZId803lSMz CIFAR10
Retrieving folder 16ICfdlUqUsDjbqlp_RE8Y9pJpn8jAdhx 20.0
Retrieving folder 122hqet8SrWdNwsv_5iN68JHOeKzLuZtA bald
Retrieving folder 1-NwxFb8xgL3Y56HXbBCqIgE9eBJ8uH6f vgg_depth_16
Retrieving folder 1U46qzbfIWLHK_wwokoQAFfsf1qzuB4AZ swa_rand_aug
Processing file 1BzykI5mCBPdEEPLqWGHgd5mkJto2s5MK activeSet.npy
Retrieving folder 1E8JSKfT6JrPTNCMgRkKivRpmj5tnJIXn vanilla
Processing file 12UC8gWR3O_ZM1BUqwe9YNftMQgJPrYTO activeSet.npy
Retrieving folder 1okQu6bg-l6-RgDy7XzffO63THTBgI52s coreset
Retrieving folder 1U24VXLrMiYmMiAJ5O2lrOd6ivX1Etxcn vgg_depth_16
Retrieving folder 131aNfbvFcN8U9QtdYqB3Zralw2ZXV8Lh swa_rand_aug
Processing file 1e1Ja48cD61H2icg7-8xMy8ISndOeWsus activeSet.npy
Retrieving folder 12cMN-93BsB77PaSu91UeLy

Downloading...
From: https://drive.google.com/uc?id=1uk2vXpnyAkvrrNJ_xETs2zWrwcZaP1Do
To: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/‪active_set_indexes‬‏/‪lSet_1‬‏/‪start_1‬‏/‪CIFAR10‬‏/20.0/‪vaal‬‏/‪vgg_depth_16‬‏/‪swa_rand_aug‬‏/activeSet.npy
100%|██████████████████████████████████████| 15.3k/15.3k [00:00<00:00, 35.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FM_9Ycs5FouQz1-e4YAl5njjKlb0wWFR
To: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/‪active_set_indexes‬‏/‪lSet_1‬‏/‪start_1‬‏/‪CIFAR10‬‏/20.0/‪vaal‬‏/‪vgg_depth_16‬‏/‪vanilla‬‏/activeSet.npy
100%|██████████████████████████████████████| 15.3k/15.3k [00:00<00:00, 40.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VhhpbtVAbm81SOLD_cdwBwmaBCQjrrkk
To: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/‪active_set_indexes‬‏/‪lSet_1‬‏/‪start_1‬‏/‪CIFAR100‬‏/20.0/‪bald‬‏/‪vgg_depth_16‬‏/‪swa_rand_aug‬‏/acti

Downloading...
From: https://drive.google.com/uc?id=1O8pAvtrU8UOXxn0J9tLgpRyt_ng9NFxq
To: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/‪results‬‏/‪best_automl_results‬‏/‪lSet_1‬‏/‪start_1‬‏/‪CIFAR100‬‏/10.0/‪vgg_depth_16‬‏/‪vanilla‬‏/‪checkpoints‬‏/vlBest_acc_33.86_model_epoch_0150.pyth
100%|████████████████████████████████████████| 957M/957M [00:36<00:00, 26.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZGWrRiEFrCtTRXPVZ5Evsvaz9dAtGP8r
To: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/‪results‬‏/‪best_automl_results‬‏/‪lSet_1‬‏/‪start_1‬‏/‪CIFAR100‬‏/10.0/‪vgg_depth_16‬‏/‪vanilla‬‏/.DS_Store
100%|██████████████████████████████████████| 6.15k/6.15k [00:00<00:00, 16.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1eAT-EWaUH8PLJqZEU3JKEADnLRlJaoH9
To: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/‪results‬‏/‪best_automl_results‬‏/‪lSet_1‬‏/‪start_1‬‏/‪CIFAR10

In [6]:
# active_set_indexes is the folder shared containing active set indexes for AL iteration of 10% to 20%
# results is the folder shared containing model checkpoints trained on 10% of CIFAR10/100 datasets 
# for both with and without strong-regularization settings.
!ls ./notebooks

Active_Sampling.ipynb	   cifar10_results.ipynb
‪active_set_indexes‬‏	   reproducible_active_set.ipynb
automl_rsb_resnet18.ipynb  ‪results‬‏
automl_rsb_vanilla.ipynb   test_cifar10_checkpoint.py.ipynb
budgetsize_exp.ipynb	   valSetexp_cifar10.ipynb


In [10]:
# Test cifar10 base classifier trained on 10% labeled data; L1 
# This is the model trained without strong-regularization

ckpt_path=r"/nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/‪results‬‏/‪best_automl_results‬‏/‪lSet_1‬‏/‪start_1‬‏/‪CIFAR10‬‏/10.0/‪vgg_depth_16‬‏/‪vanilla‬‏/‪checkpoints‬‏/vlBest_acc_70.47999824523926_model_epoch_0144.pyth"
config_path=r"/nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/‪results‬‏/‪best_automl_results‬‏/‪lSet_1‬‏/‪start_1‬‏/‪CIFAR10‬‏/10.0/‪vgg_depth_16‬‏/‪vanilla‬‏/config.yaml"

print(ckpt_path)

assert os.path.isfile(ckpt_path)
assert os.path.isfile(config_path)

/nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/‪results‬‏/‪best_automl_results‬‏/‪lSet_1‬‏/‪start_1‬‏/‪CIFAR10‬‏/10.0/‪vgg_depth_16‬‏/‪vanilla‬‏/‪checkpoints‬‏/vlBest_acc_70.47999824523926_model_epoch_0144.pyth


## Update the config file

* Kindly check the LSET_PATH, USET_PATH, VALSET_PATH points to the correct dataset indexes.

* Modify OUT_DIR as per your choice - the program writes the test set accuracy in this path.

* Modify SAMPLING_FN: as per desired AL sampling function

## After update

In [11]:
# We updated sampling function to uncertainty and out_directory to "check_results"
# We can verify it by simpling reading the YAML file.

import yaml

with open(config_path) as file:
    config_list = yaml.load(file, Loader=yaml.FullLoader)

    print("AL Sampling function: ",config_list["ACTIVE_LEARNING"]["SAMPLING_FN"])
    print("Out_dir: ", config_list["OUT_DIR"])

AL Sampling function:  uncertainty
Out_dir:  /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/check_results


In [15]:
def save_plot_values(temp_arrays, temp_names, cfg, isParallel=True, saveInTextFormat=False):
    """ Saves arrays provided in the list in npy format """
    #return if not master process
    if isParallel:
        if not du.is_master_proc():
            return

    for i in range(len(temp_arrays)):
        temp_arrays[i] = np.array(temp_arrays[i])
        if not os.path.exists(cfg.OUT_DIR):
            os.makedirs(cfg.OUT_DIR)
        if saveInTextFormat:
            print(f"Saving {temp_names[i]} at {cfg.OUT_DIR+temp_names[i]}.txt in text format!!")
            np.savetxt(cfg.OUT_DIR+temp_names[i]+".txt", temp_arrays[i], fmt="%d")
        else:
            print(f"Saving {temp_names[i]} at {cfg.OUT_DIR+temp_names[i]}.npy in numpy format!!")
            np.save(cfg.OUT_DIR+temp_names[i]+".npy", temp_arrays[i])

def active_sampling(cfg):

    model_path = cfg.ACTIVE_LEARNING.MODEL_LOAD_DIR
    model = model_builder.build_model(cfg, active_sampling=cfg.ACTIVE_LEARNING.ACTIVATE, isDistributed=False)

    checkpoint = torch.load(model_path, map_location='cpu')
    
    if 'model_state' in checkpoint:
        model.load_state_dict(checkpoint['model_state'])
    else:
        isModuleStrPresent=False
        #remove module
        for k in checkpoint.keys():
            if k.find("module.") == -1:
                continue
            isModuleStrPresent=True
            break
    
        if isModuleStrPresent:
            print("Loaded checkpoint contains module present in keys.")
            print("So now removing 'module' strings")
            #remove module strings
            from collections import OrderedDict
            new_ckpt_dict = OrderedDict()
            for k,v in checkpoint.items():
                tmp_key = k.replace("module.","")
                new_ckpt_dict[tmp_key] = v
            
            checkpoint = copy.deepcopy(new_ckpt_dict)
            print("Done!!")

        model.load_state_dict(checkpoint)
    model.cuda(torch.cuda.current_device())

    #load dataset
    #We can load noAug dataset as we just need datapoints (not their labels)
    #for uSet and lSet
    #However if you make use of labels in active sampling then do not load dataset references
    #rather pass it from main otherwise for noisy experiments results will not be consistent
    if cfg.TRAIN.DATASET in ["CIFAR10", "CIFAR100"]:
        #No need to pass randaug true here as for AL we never intent to use data augmentations
        dataObj = custom_Data(dataset=cfg.TRAIN.DATASET)
        
        dataObj.eval_mode = True #To load the data w/o any aug
        noAugDataset, _ = dataObj.getDataset(save_dir=cfg.TRAIN_DIR, isTrain=True, isDownload=True)
        
        lSet, uSet, valSet = dataObj.loadPartitions(lSetPath=cfg.ACTIVE_LEARNING.LSET_PATH, \
                uSetPath=cfg.ACTIVE_LEARNING.USET_PATH, valSetPath = cfg.ACTIVE_LEARNING.VALSET_PATH)

    
    print("Perform {} sampling through subprocess".format(cfg.ACTIVE_LEARNING.SAMPLING_FN))

    if cfg.ACTIVE_LEARNING.SAMPLING_FN.lower() in ["vaal","vaal_minus_disc"]:
        return vaal_sampling_util(cfg, dataObj, debug=True)
    else:
        activelearning = ActiveLearning(dataObj=dataObj, cfg=cfg)

    print("========BEFORE==========")
    print("len(uSEt): ",len(uSet))
    print("len(lSEt): ",len(lSet))
    print("==================")    
        
    # with torch.no_grad():
    activeSet, uSet = activelearning.sample_from_uSet(clf_model=model, \
        lSet=lSet, uSet=uSet, trainDataset=noAugDataset)
    
    lSet  = np.append(lSet, activeSet)
    assert len(set(valSet) & set(uSet)) == 0,"Intersection is not allowed between validationset and uset"
    assert len(set(valSet) & set(lSet)) == 0,"Intersection is not allowed between validationset and lSet"
    assert len(set(uSet) & set(lSet)) == 0,"Intersection is not allowed between uSet and lSet"
    print("After including activeSet -- len(lSet): {} and len(uSet): {}".format(len(lSet), len(uSet)))
    
    #Save numpy arrays
    print('saving pickle values...')
    save_plot_values([lSet, uSet,activeSet], ['lSet','uSet','activeSet'], cfg, isParallel=False)
    print('saved!!')
    #Save index sets in txt formats
    print('saving text values...')
    save_plot_values([lSet, uSet,activeSet], ['lSet','uSet','activeSet'], cfg, isParallel=False, saveInTextFormat=True)
    print('saved!!')
    print("======AFTER AL============")
    print("ActiveSet: ",len(activeSet))
    print("len(uSet): ",len(uSet))
    print("len(lSet): ",len(lSet))


In [16]:
cfg.merge_from_file(config_path)
# Setup model path to perform AL sampling
cfg["ACTIVE_LEARNING"]["MODEL_LOAD_DIR"]=ckpt_path
print("Budget size for AL sampling: ", cfg["ACTIVE_LEARNING"]["BUDGET_SIZE"])
active_sampling(cfg)

Budget size for AL sampling:  5000
########### cfg model type: vgg
Dataset is augmented
--------------------------------------
Preprocess Operations Selected ==>  [ToTensor()]
--------------------------------------
Files already downloaded and verified


uSet Activations:   1%|          | 18/2500 [00:00<00:14, 175.27it/s]

Perform uncertainty sampling through subprocess
========BEFORE==========
len(uSEt):  40000
len(lSEt):  5000
len(uSetLoader): 2500


uSet Activations: 100%|██████████| 2500/2500 [00:13<00:00, 183.00it/s]


u_ranks.shape: (40000,)
===========In custom dum cfg===========
temp_cfg.OUT_DIR: /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/check_results/, temp_cfg.CFG_DEST: config.yaml
After including activeSet -- len(lSet): 10000 and len(uSet): 35000
saving pickle values...
Saving lSet at /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/check_results/lSet.npy in numpy format!!
Saving uSet at /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/check_results/uSet.npy in numpy format!!
Saving activeSet at /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/check_results/activeSet.npy in numpy format!!
saved!!
saving text values...
Saving lSet at /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/check_results/lSet.txt in text format!!
Saving uSet at /nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/check_results/uS

## Ensure Reproducibility by comparing active sets

Lets compare the active sets we shared Vs the ones which we just extracted by running the above cell for uncertainty sampling method.

In [17]:
uncertainty_indexes=[]
fp = open(os.path.join(cfg.OUT_DIR,"activeSet.txt"))

for line in fp.readlines():
    line = line.strip()
    uncertainty_indexes.append(line)

print("len: ", len(uncertainty_indexes))
print(uncertainty_indexes[:10]) #first 10 indexes with highest UC

len:  5000
['39872', '26346', '9465', '19703', '5931', '25831', '48718', '40927', '10782', '923']


In [18]:
# Already shared active set indexes

reported_uncertainty_indexes=[]
fpath="/nfs/users/ext_prateek.munjal/projects/active_learning_codebase_automl/notebooks/active_set_indexes/lSet_1/start_1/CIFAR10/20.0/uncertainty/vgg_depth_16/vanilla/activeSet.npy"

np_array = np.load(fpath, allow_pickle=True)
np_array = [str(x) for x in np_array]

print("len: ", len(np_array))
print(np_array[:10]) #first 10 indexes with highest UC

len:  5000
['39872', '26346', '9465', '19703', '5931', '25831', '48718', '40927', '10782', '923']


In [19]:
# TO see if they are exactly same we will see simply check if the length of their intersection is same as len of both sets.

intersect = set(np_array).intersection(set(uncertainty_indexes))

if len(intersect)==len(np_array) and len(intersect)==len(uncertainty_indexes):
    print("Both index lists are same!")
else:
    print("SOme issue")


Both index lists are same!
